# EJERCICIO BÁSICO

Las funciones que se ocupan del cálculo de los estadísticos pedidos se encuentras en el fichero funciones.py. Se decidió hacer esto para mayor limpieza en la presentación de los ejercicios. Las funciones estan debidamente documentadas en el archivo .py.

In [1]:
from pyspark import SparkContext
import numpy as np
import funciones as udf

package "funciones" succesfully loaded


Primero se crea la sesion de spark y se declara la ruta que contiene los archivos csv de alturas.

In [2]:
sc = SparkContext(master = 'local[*]')
main_path = '/media/daniel/Seagate Basic/spark_practica1' # path donde se encuantran los csv

In [19]:
def limpia_alturas(rdd, col=0, sep=','):
    
    # descripcion
    #----------------
    
    #limpia los datos de alturas. Las tareas de la pipeline son: seaparar las lineas del RDD por el separador argumento
    # pasar a centímetros (quitando los puntos), eliminar los números negativos y agregar una cifra a las 
    # alturas que estando en metros solo tenian un decimal.
    
    # argumentos:
    #----------------
    
    #<rdd> | tipo: pyspark.rdd 
    #<col> | tipo: int | descriptivo: columna donde estan las alturas
    #<sep> | tipo: str | seapardor del fichero de texto
    
    # resultado: 
    #----------------
    
    #<result> | tipo: pyspark.rdd  | descriptivo: rdd separado por lineas
    
    rdd_limpio = udf.preprocesa_rdd(rdd)\
                     .map(lambda x: x[:col]+[x[col].replace('.', '')]+x[col+1:])\
                     .filter(lambda x: '-' not in x[col])\
                     .map(lambda x: x[:col]+[float(x[col]+'0')]+x[col+1:] if len(x[col])<3 else x)
        
    return rdd_limpio

# PRIMERA PARTE

Se lee el archivo y cada linea se guarda como un string.

In [15]:
alturas = f'{main_path}/alturas_v0.csv'
rdd_alturas = sc.textFile(alturas)

Despues se preprocesan los datos, esto es: 
- Se eliminan los datos negativos
- Se pasan todas las observaciones a centímetros.
- Se añade un 0 a las observaciones en metros dadas con una única cifra significativa.

Además, las funciones que obtienen los resultados pedidos siempre necesitan una columna por la que agregar. Para eso, se crea la clave 'global' para todas las observaciones. 

In [16]:
rdd_alturas_limpio = limpia_alturas(rdd_alturas).map(lambda x: ['global']+ x)

In [20]:
medias = udf.summary(rdd_alturas_limpio, features=[1], agg=[0], std=['mean'])
medias.take(2)

[('global', (168.93825222396651,))]

In [21]:
std = udf.summary(rdd_alturas_limpio, features=[1], agg=[0], std=['std'])
std.take(2)

[('global', (7.8340557833708955,))]

# SEGUNDA PARTE

En esta segunda parte, se calculan los estadísticos agregando por sexo. Se repite el proceso anteriormente descrito y para el cálculo se usan las funciones desarrolladas.

In [22]:
alturas2 = f'{main_path}/alturas.csv'
rdd_alturas2 = sc.textFile(alturas2)

rdd_alturas_limpio2 = limpia_alturas(rdd_alturas2, col=1)

In [23]:
medias = udf.summary(rdd_alturas_limpio2, features=[1], agg=[0], std=['mean'])
medias.take(2)

[('H', (173.46075433231397,)), ('M', (164.16774193548386,))]

In [24]:
std = udf.summary(rdd_alturas_limpio2, features=[1], agg=[0], std=['std'])
std.take(2)

[('H', (7.53122883663738,)), ('M', (4.684891325217995,))]